<a href="https://colab.research.google.com/github/GGpark1/User-Log-data-analytics/blob/main/Collaborative_filltering_computers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- 구매 기록이 한번도 없는 상품은 추천 목록에서 제외 -> 희소문제를 해결하기 위함 
  - 전자 제품 군의 14,950개의 상품 중에서 한번도 팔리지 않은 상품은 약 10,000개로 분석에 포함될 시 희소문제를 일으킬 가능성이 높음

- 편향 보정을 위해 스마트폰을 구매한 일부 비율을 뺄까?

## 구매 횟수 필터 없는 추천 시스템

In [1]:
from pandas.io import gbq

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import svds
#pd.options.display.float_format = "{:,.2f}".format
import warnings
warnings.simplefilter("ignore")

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'appliances%' AND event_type = 'purchase'
GROUP BY user_id, product_id
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [ ]:
df_ratings = df

In [ ]:
base = df_ratings['purchase_count'].mean()
base

1.1954411413086794

In [25]:
sql = """SELECT product_id, brand, category_code FROM `empirical-weft-350709.CP2_ggpark.log_data`
WHERE event_type = 'purchase' AND category_code LIKE 'computer%'
GROUP BY product_id, brand, category_code"""
project_id = 'empirical-weft-350709'

df_product = pd.read_gbq(sql, project_id=project_id, dialect='standard')

In [26]:
product_data = df_product
product_data.head()

,product_id,brand,category_code
0,25900012,midea,computers.components.cooler
1,6600702,crucial,computers.components.memory
2,11600299,microlab,computers.desktop
3,1600558,canon,computers.peripherals.printer
4,1500440,epson,computers.peripherals.printer


In [ ]:
df_fin = pd.pivot_table(df, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')

In [ ]:
df_fin

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
384989212,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
429474454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433754231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
440756116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450756413,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566221288,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566227497,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566235555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_base_line = df_fin.fillna(base)

In [ ]:
df_final = df_fin.fillna(0)

In [ ]:
df_final['total'] = df_final.sum(axis=1)
df_filtered = df_final.loc[df_final['total'] > 1]
df_filtered.drop(columns='total', inplace = True)

df_filtered

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481341715,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
494312717,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
500246310,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512363712,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566094079,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566122364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566144303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
matrix = df_filtered.values
user_ratings_mean = np.mean(matrix, axis = 1)

In [ ]:
user_ratings_mean_reshape = user_ratings_mean.reshape(-1, 1)

In [ ]:
matrix_user_mean = matrix - user_ratings_mean_reshape

In [ ]:
pd.DataFrame(matrix_user_mean, columns = df_filtered.columns, index = df_filtered.index).head()

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,...,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240,-0.001240
481341715,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413
494312717,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413
500246310,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413
512363712,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,...,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413,-0.000413


In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
U, sigma, Vt = svds(matrix_user_mean, k = 200)

In [ ]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(12561, 200)
(200,)
(200, 4838)


In [ ]:
sigma = np.diag(sigma)

In [ ]:
sigma.shape

(200, 200)

In [ ]:
sigma[0]

array([9.32635077, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [ ]:
dot_matrix = np.dot(U, sigma)

In [ ]:
df_svd_predicted_purchase = np.dot(dot_matrix, Vt)

In [ ]:
df_svd_predicted_purchase = df_svd_predicted_purchase + user_ratings_mean_reshape

In [ ]:
df_svd_preds = pd.DataFrame(df_svd_predicted_purchase, columns = df_filtered.columns, index = df_filtered.index)
df_svd_preds.head()

product_id,2400103,2400109,2400163,2400194,2400216,2400257,2400260,2400267,2400282,2400303,...,46800030,46800031,51000001,51000002,51000006,51000010,51000016,51000027,51000028,51000029
user_id,,,,,,,,,,,,,,,,,,,,,
475786534,0.000391,0.000394,0.000394,0.000380,0.000801,0.000394,0.000378,0.000389,0.000396,0.000440,...,0.000382,0.000394,0.001167,-0.002678,0.000394,0.000389,0.000394,0.000394,0.000394,0.000396
481341715,0.000198,0.000199,0.000199,0.000200,0.000103,0.000199,0.000195,0.000191,0.000205,0.000197,...,0.000196,0.000199,-0.000089,0.000062,0.000199,0.000197,0.000199,0.000199,0.000199,0.000191
494312717,-0.000121,-0.000114,-0.000114,-0.000115,-0.002820,-0.000114,-0.000129,-0.000115,-0.000143,-0.000133,...,-0.000133,-0.000114,0.000070,-0.000059,-0.000114,-0.000117,-0.000114,-0.000114,-0.000114,-0.000175
500246310,-0.000094,-0.000094,-0.000094,-0.000096,-0.001699,-0.000094,-0.000095,-0.000092,-0.000116,-0.000086,...,-0.000103,-0.000094,0.000027,-0.000017,-0.000094,-0.000095,-0.000094,-0.000094,-0.000094,-0.000072
512363712,-0.000245,-0.000228,-0.000228,-0.000252,-0.000253,-0.000228,-0.000272,-0.000268,-0.000275,-0.000429,...,-0.000260,-0.000228,0.000033,0.000024,-0.000228,-0.000246,-0.000228,-0.000228,-0.000228,-0.000298


In [22]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df, ori_ratings_df, num_recommendations=5):
    
    #현재는 index로 적용이 되어있으므로 user_id - 1을 해야함.
    user_row_number = user_id 
    
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 영화 데이터 정렬 -> 영화 평점이 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    user_data = ori_ratings_df[ori_ratings_df.user_id == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터를 합친다. 
    user_history = user_data.merge(ori_movies_df, on = 'product_id').sort_values(['purchase_count'], ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터를 추출
    recommendations = ori_movies_df[~ori_movies_df['product_id'].isin(user_history['product_id'])]
    # 사용자의 영화 평점이 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = recommendations.merge( pd.DataFrame(sorted_user_predictions).reset_index(), on = 'product_id')
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_row_number: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
                      

    return user_history, recommendations

In [ ]:
already_rated, predictions = recommend_movies(df_svd_preds, 566235555, product_data, df_ratings, 10)

In [ ]:
predictions

,product_id,brand,category_code,Predictions
937,2501751,artel,appliances.kitchen.oven,0.261552
247,2500859,asel,appliances.kitchen.oven,0.057857
447,2900561,None,appliances.kitchen.microwave,0.055849
446,2900561,midea,appliances.kitchen.microwave,0.055849
1774,3601551,midea,appliances.kitchen.washer,0.032020
938,2501796,artel,appliances.kitchen.oven,0.022036
1353,2501909,dauscher,appliances.kitchen.oven,0.021331
1346,3200550,dauscher,appliances.kitchen.meat_grinder,0.017683
4391,29100058,maxwell,appliances.personal.massager,0.016791
1394,3701387,dauscher,appliances.environment.vacuum,0.015864


In [23]:
def evaluate(sparse_matrix, pred_matrix):
        idx, jdx = sparse_matrix.to_numpy().nonzero()
        ys, preds = [], []
        for i, j in zip(idx, jdx):
            ys.append(sparse_matrix.iloc[i, j])
            preds.append(pred_matrix.iloc[i, j])

        error = mean_squared_error(ys, preds)
        return np.sqrt(error)

In [ ]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_base_line)

1.25495616539933

In [ ]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered, df_svd_preds)

1.065752985150627

## 구매 횟수 필터 있는 추천 시스템

In [4]:
sql = """SELECT user_id, product_id, count(*) AS purchase_count
FROM empirical-weft-350709.CP2_ggpark.log_data
WHERE category_code LIKE 'computer%' AND event_type = 'purchase'
GROUP BY user_id, product_id
HAVING purchase_count <= 5
ORDER BY purchase_count DESC"""
project_id = 'empirical-weft-350709'

df_limit = pd.read_gbq(sql, project_id=project_id, dialect='standard')

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=RMzDM3yNCdudSICpOQ16rndt5O1N2S&prompt=consent&access_type=offline
Enter the authorization code: 4/1AX4XfWiJRKRV_wm7qqdJBeXSZ2A04NWUEiU0AekAD3MXao7MdIkvrjkC0tI


In [5]:
df_limit.sort_values('purchase_count')

,user_id,product_id,purchase_count
10647,515629946,30900644,1
14201,513223565,1600614,1
14200,513439030,1600600,1
14199,516048765,1500463,1
14198,534073454,1500463,1
...,...,...,...
74,549333671,6701171,5
75,537329835,1500021,5
76,512472114,1307136,5
78,513053457,1307068,5


In [6]:
df_ratings_limit = df_limit
mean = df_ratings_limit['purchase_count'].mean()

In [7]:
df_fin_limit = pd.pivot_table(df_limit, index = ['user_id'], columns = ['product_id'], values = 'purchase_count')
df_fin_limit

product_id,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
user_id,,,,,,,,,,,,,,,,,,,,,
418115333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
420935067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
445228646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
465723791,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
479752015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
566196738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566201318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
566204832,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_final_limit = df_fin_limit.fillna(0)

In [9]:
df_base_line_limit = df_fin_limit.fillna(mean)

In [10]:
df_final_limit['total'] = df_final_limit.sum(axis=1)
df_filtered_limit = df_final_limit.loc[df_final_limit['total'] > 1]
df_filtered_limit.drop(columns='total', inplace = True)

df_filtered_limit

product_id,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
user_id,,,,,,,,,,,,,,,,,,,,,
445228646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479752015,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512363815,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512364184,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
512364323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565970301,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566011852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
566015450,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
matrix_limit = df_filtered_limit.values
user_ratings_mean_limit = np.mean(matrix_limit, axis = 1)

In [12]:
user_ratings_mean_reshape_limit = user_ratings_mean_limit.reshape(-1, 1)

In [13]:
matrix_user_mean_limit = matrix_limit - user_ratings_mean_reshape_limit

In [14]:
U_limit, sigma_limit, Vt_limit = svds(matrix_user_mean_limit, k = 1000)

In [15]:
sigma_limit = np.diag(sigma_limit)

In [16]:
dot_matrix_limit = np.dot(U_limit, sigma_limit)

In [17]:
df_svd_predicted_purchase_limit = np.dot(dot_matrix_limit, Vt_limit)

In [18]:
df_svd_predicted_purchase_limit = df_svd_predicted_purchase_limit + user_ratings_mean_reshape_limit

In [19]:
df_svd_preds_limit = pd.DataFrame(df_svd_predicted_purchase_limit, columns = df_filtered_limit.columns, index = df_filtered_limit.index)
df_svd_preds_limit.head()

product_id,1303049,1303355,1304106,1304297,1304346,1304392,1304409,1304410,1304411,1304438,...,45500013,45500014,45500015,45500019,45500021,45500022,45500027,45500028,45500029,45500030
user_id,,,,,,,,,,,,,,,,,,,,,
445228646,-0.000003,0.000002,-0.000003,0.000003,-0.000003,0.000017,-0.000005,0.000013,0.000129,-0.000012,...,-0.000003,-0.000002,0.000009,0.000002,0.000016,0.000013,-0.000012,-0.000003,0.000018,-0.000006
479752015,-0.000039,-0.000006,-0.000039,0.000002,-0.000039,-0.000241,0.000090,-0.000051,-0.001977,0.000379,...,-0.000039,0.000046,0.000017,0.000023,-0.000020,-0.000014,-0.000030,-0.000039,0.000085,0.000214
512363815,0.000065,-0.000078,0.000065,-0.000338,0.000065,-0.000468,1.001212,0.005131,-0.006477,-0.000633,...,0.000065,0.000308,-0.000348,-0.005479,-0.002027,-0.000127,-0.000223,0.000065,-0.017498,-0.006246
512364184,0.000002,-0.000007,0.000002,0.000005,0.000002,-0.001009,0.000029,-0.000015,0.000303,-0.000047,...,0.000002,-0.000034,0.000002,0.000060,-0.000052,-0.000047,0.000007,0.000002,-0.000081,0.000135
512364323,0.000128,0.000121,0.000128,0.000085,0.000128,0.000097,0.000455,-0.000295,0.119358,0.000181,...,0.000128,-0.000041,0.000017,0.002323,-0.000622,-0.000020,0.000859,0.000128,0.000186,-0.016059


In [27]:
already_rated, predictions = recommend_movies(df_svd_preds_limit, 512363815, product_data, df_ratings_limit, 10)

In [28]:
predictions

,product_id,brand,category_code,Predictions
1202,1306571,acer,computers.notebook,0.142998
607,1306832,lenovo,computers.notebook,0.086788
2508,17800075,None,computers.desktop,0.069365
404,1701515,lg,computers.peripherals.monitor,0.069365
1750,11600095,defender,computers.desktop,0.058644
1046,11600186,defender,computers.desktop,0.038518
1862,1701009,samsung,computers.peripherals.monitor,0.034265
2355,6600944,zeppelin,computers.components.memory,0.031786
619,6801092,samsung,computers.components.hdd,0.022437
53,9200673,ritmix,computers.peripherals.keyboard,0.020554


In [29]:
# Base Line -> 예측 데이터를 평균으로 채워 넣음
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_base_line_limit)

0.8508147409811612

In [30]:
# 행렬 분해 방식으로 도출한 예측 데이터
# Mean_squared_error
# 구매 횟수에 필터를 걸지 않은 결과

evaluate(df_filtered_limit, df_svd_preds_limit)

0.22831300945010938